In [ ]:
%pip install datasets transformers
%pip install torch
%pip install accelerate

from datasets import load_dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Prepare

In [ ]:
dataset = load_dataset(
    "takala/financial_phrasebank", "sentences_allagree", split="train"
)

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label'],
    num_rows: 2264
})


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
).to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    eval_strategy="epoch",  # evaluation strategy to use
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    num_train_epochs=3,  # number of training epochs
    weight_decay=0.01,  # strength of weight decay
    report_to=[]
)


In [ ]:
trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=tokenized_dataset,  # training dataset
    eval_dataset=tokenized_dataset,  # evaluation dataset
)


### Train

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.101529


Epoch,Training Loss,Validation Loss
1,No log,0.101529
2,No log,0.043177
3,No log,0.028472


TrainOutput(global_step=426, training_loss=0.18587694481504916, metrics={'train_runtime': 817.1525, 'train_samples_per_second': 8.312, 'train_steps_per_second': 0.521, 'total_flos': 1787066333208576.0, 'train_loss': 0.18587694481504916, 'epoch': 3.0})

### Save Model

In [22]:
model.save_pretrained("financial_phrasebank_model")

### Predict

In [30]:
test_text = [
    "The company's profit has increased significantly this quarter.",
    "The increase in costs negatively affected the revenue.",
    "The company's performance remained stable."
]

test_text_token = tokenizer(test_text, padding=True, truncation=True, return_tensors="pt").to(
    device
)

In [31]:
output = model(**test_text_token)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1653, -2.3997,  4.1295],
        [ 3.6392, -2.2372, -0.8891],
        [-1.4240, -2.4179,  3.3680]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [36]:
labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

preds = torch.argmax(output.logits, dim=1)

for i, pred in enumerate(preds):
    print(f"[{labels[pred.item()]}] {test_text[i]}")

[Positive] The company's profit has increased significantly this quarter.
[Negative] The increase in costs negatively affected the revenue.
[Positive] The company's performance remained stable.
